## Erode the surface during CRN production/decay
<br>
  
#### Syntax
`conc10Be, conc14C = crnErodeFxn(conc10Be, conc14C, erode_z, CosmoPars)` <br>
#### Input 
`conc10Be` : vector of size len(CRNz) with spacing dz containing the concentration of <sup>10Be </sup>(at g-1). <br>
`conc14C` : vector of size len(CRNz) with spacing dz containing the concentration of <sup>14C </sup>(at g-1). <br>
`erode_z` : depth of the annual erosion event (cm). <br>
`CosmoPars` : dictionary of parameters for the cosmogenic nuclide calculations. <br>
#### Variables Used
##### Cosmo Pars
`CRNz` : : depth vector of len(max_depths / dz) <br>
#### Output
`conc10Be` : vector of size len(CRNz) with spacing dz containing <sup>10</sup>Be concentrations after this year (at g-1).<br>
`conc14C` : vector of size len(CRNz) with spacing dz containing <sup>14</sup>C concentrations after this year (at g-1).<br>
#### Notes
**Date of Creation:** 1. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>
04.03.2022 : Renamed to `crnErodeFxn` from `crnErodeFxnmtest` for publication.

In [5]:
def crnErodeFxn(conc10Be, muon10Be, spall10Be, conc14C, erode_z, CosmoPars):

    conc10Be, muon10Be, spall10Be = Be10AnnualFxn(conc10Be,muon10Be,spall10Be, CosmoPars) # calculate how much Be-10 accumulates
    conc14C = C14AnnualFxn(conc14C, CosmoPars)
    CRNz = CosmoPars['CRNz']
    B_Be = CosmoPars['B_Be']
    rock_rho = CosmoPars['rock_rho']
    L3 = CosmoPars['L3 Be']
    
    old_depths = CRNz[CRNz > erode_z] # return only depths which are deeper than the erosion event
    minus = len(CRNz) - len(old_depths) # determine the number of nodes to remove from the 'surface'
    
    if minus > 0:
        
        # define the 10Be interpolation function on the remaining nodes
        sec_Bemfxn = interp1d(x = (old_depths - erode_z), 
                             y = muon10Be[minus:],
                             kind = 'linear',
                             fill_value = "extrapolate")
        
        sec_Besfxn = interp1d(x = (old_depths - erode_z), 
                             y = spall10Be[minus:],
                             kind = 'linear',
                             fill_value = "extrapolate")
        
        sec_Becfxn = interp1d(x = (old_depths - erode_z), 
                             y = conc10Be[minus:],
                             kind = 'linear',
                             fill_value = "extrapolate")
        
        # interpolate the concentrations of the nodes which are now at the surface
        muon10Be[:-minus] = sec_Bemfxn(x = CRNz[:-minus])
        spall10Be[:-minus] = sec_Besfxn(x = CRNz[:-minus])
        conc10Be[:-minus] = sec_Becfxn(x = CRNz[:-minus])
        # replace all nodes which were greater than max_depth[1] with 0.0 at g-1 concentration
        conc10Be[-minus:] = [0] * minus
        #muon10Be[-minus:] = [0] * B_Be * np.e**(-rock_rho * CRNz[i] / L3)
        muon10Be[-minus:] = [B_Be * np.e**(-rock_rho * CRNz[-1] / L3)] * minus
        spall10Be[-minus:] = [0] * minus
        
        # define the 14C interpolation function on the remaining nodes
        sec_Cfxn = interp1d(x = (old_depths - erode_z), 
                            y = conc14C[minus:], 
                            kind = 'linear',
                            fill_value = "extrapolate")
        # interpolate the concentrations of the nodes which are now at the surface
        conc14C[:-minus] = sec_Cfxn(x = CRNz[:-minus])
        # replace all nodes which were greater than max_depth[1] with 0.0 at g-1 concentration
        conc14C[-minus:] = [0] * minus
    
    else:
        conc10Be = conc10Be
        conc14C = conc14C
            
    return conc10Be, muon10Be, spall10Be, conc14C